In [1]:
import pandas as pd
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
from config import *
import fitz
from tqdm import tqdm

In [2]:
OUTPUT_FOLDER = "../data/output"
output_files = os.listdir(OUTPUT_FOLDER)

In [44]:
compliant_files = []

for filename in tqdm(output_files):
    if not filename.endswith("_highlighted.pdf"):
        continue
        
    pdf_path = os.path.join(OUTPUT_FOLDER, filename)
    
    # Read the PDF and check for highlights
    doc = fitz.open(pdf_path)
    found_colors = set()
    has_highlights = False
    
    # Check each page for highlights
    for page in doc:
        highlights = page.annots()
        
        for highlight in highlights:
            if highlight.type[0] != fitz.PDF_ANNOT_HIGHLIGHT:
                continue
                
            has_highlights = True
            color = highlight.colors["stroke"]
            # Convert color tuple to match CRITERIA_COLORS format
            color_tuple = (round(color[0], 1), round(color[1], 1), round(color[2], 1))
            
            # Check if highlight has a comment starting with "YES"
            comment = highlight.info.get("content", "")
            if comment.startswith("YES"):
                # Add color to set if it's one of the first 4 criteria colors
                if color_tuple in list(CRITERIA_COLORS.values())[:4]:
                    found_colors.add(color_tuple)
                
    # Check if all 4 colors were found
    if has_highlights and len(found_colors) == 4:
        compliant_files.append(filename)
        
    doc.close()

print(f"Found {len(compliant_files)} compliant files:")
for file in compliant_files:
    print(f"- {file}")


100%|██████████| 211/211 [00:05<00:00, 35.44it/s]

Found 1 compliant files:
- Basu, Wagner, Sewpaul, Reddy, Davies_The Lancet Global Health_2019_highlighted.pdf


In [34]:
df = pd.read_csv("../data/Exported Items.csv")
df.head(1)

,Key,Item Type,Publication Year,Author,Title,Publication Title,ISBN,ISSN,DOI,Url,...,Programming Language,Version,System,Code,Code Number,Section,Session,Committee,History,Legislative Body
0,7Z3S4GYD,journalArticle,2021.0,"Jayasekera, Jinani; Sparano, Joseph A.; O'Neil...",Development and Validation of a Simulation Mod...,Journal of Clinical Oncology,NaN,0732-183X,10.1200/JCO.21.00651,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC8...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [42]:
df['Manual Tags'].str.contains('obj2_include').sum()

np.int64(75)